# Introduction to Markov Networks



## CSCI E-83
## Stephen Elston

in the previous lesson we explored directed Bayes networks (BNs). BNs are directed acyclic graphs (DAGs) which can represent causality with many probability distributions. Further, BNs can express independencies of probability distributions. However, there is no unique representation of independencies. Many BNs can exhibit the same independency structure, but with different causal relationships. 

Now, we will turn our attention to another class of probabilistic graphical models known as **Markov networks** or **Markov graphical models**. Markov graphic models are not directed graphs. Yet, like BNs, Markov graphical models can represent probability distributions including the independencies. In fact, as you will see, there are methods to map from BNs to Markov networks and back. 


## Graph separation criteria

You can develop a useful view of the relationship between BNs and Markov networks with the **D-separation** criteria, **directed-separation** criteria. First, we need to introduce a defintion:  

> **Definition:** An **imorality** in a directed graph $G$ occurs where either; a) there is a directed edge between $X$ and $Y$, or b) $X$ and $Y$ are both parents of the same note $Z$. 

This leads to a concept of a **moralized graph** that relates a directed BN to an undirected Markov network:

> **Definition:** A **moral graph**, $M(G)$, of a BN structure, $G$ over $X$ is the **undirected graph** over $X$ that contains an undirected edge between $X$ and $Y$ if; a) there is a directed edge between $X$ and $Y$, or b) $X$ and $Y$ are both parents of the same note $Z$.

This leads us to a correllary of relating the independencies of the directed BN to the independencies of a MN:

> **Corollary:** Given a distribution $P_B$ such that $B$ is a parameterization on a graph $G$, then $M(G)$ is an I-map for $P_B$.

What does all of this mean? 

<img src="img/MoralizedGraph.JPG" alt="Drawing" style="width:600px; height:200px"/>
<center> **Example of Graph Moralization** </center>